<H2>手書き文字認識を行うNeural Networkのレシピ</H2><br><H3>中原作成</H3>
ニューラルネットワークを使った画像認識のレシピです。今回はチュートリアルなどでよく使われている手書き文字データセットMNISTを学習してモデルを作成し、テスト画像を入力してその認識精度を確認します。

Chainer 4.5.0<BR>
CuPy 4.5.0<BR>
CUDA 9.2<BR>
  

<H2>環境のセットアップ（TSUBAMEでは不要？)</H2>

In [104]:
# Check GPU status
!nvidia-smi

Mon Jul  1 03:44:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |    467MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [105]:
# Install Chainer v4.5.0, Cupy v4.5.0 and CUDA
!curl https://colab.chainer.org/install | CHAINER_VERSION="==4.5.0" CUPY_VERSION="==4.5.0" sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  16989      0 --:--:-- --:--:-- --:--:-- 16989
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
+ pip install -q cupy-cuda100 ==4.5.0 chainer ==4.5.0
  ERROR: Could not find a version that satisfies the requirement cupy-cuda100==4.5.0 (from versions: 5.1.0, 5.2.0, 5.3.0, 5.4.0, 6.0.0b1, 6.0.0b2, 6.0.0b3, 6.0.0rc1, 6.0.0, 6.1.0, 7.0.0a1, 7.0.0b1)
ERROR: No matching distribution found for cupy-cuda100==4.5.0


<H3>必要なライブラリのインポート</H3>

In [0]:
# Import libraries
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from chainer.dataset import concat_examples
from chainer.backends.cuda import to_cpu
import numpy as np
import time

In [107]:
# Check environment
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)
import chainer
chainer.print_runtime_info()

GPU availability: True
cuDNN availablility: True
Platform: Linux-4.14.79+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 5.4.0
NumPy: 1.16.4
CuPy:
  CuPy Version          : 5.4.0
  CUDA Root             : /usr/local/cuda
  CUDA Build Version    : 10000
  CUDA Driver Version   : 10000
  CUDA Runtime Version  : 10000
  cuDNN Build Version   : 7301
  cuDNN Version         : 7301
  NCCL Build Version    : 2402
  NCCL Runtime Version  : 2402
iDeep: 2.0.0.post3


<H3>ニューラルネットワークの定義</H3>

In [0]:
# Define Binary DNN (3-layers), note that, we use a mixed-precision (Binary + Float32) neural network
# To preserve a recognition accuracy. If you want learn more, see my FPGA'18 paper!
class MNIST(chainer.Chain):

    def __init__(self,n_mid_units=100,n_out=10):
        super(MNIST, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(784, n_mid_units)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

<H3>ニューラルネットワークのパラメータと学習の設定</H3>

In [0]:
# Setup parameters
args_gpu=0 # 0: GPU (strongly recommendded), -1: CPU
args_unit=100 # The number of neurons in hidden layers
args_batchsize=100
args_epoch=3

<H3>定義したニューラルネットワークをメモリに読み込み、学習させる準備を行います。</H3>

In [110]:
# Printout parameters
print('GPU: {}'.format(args_gpu))
print('# unit: {}'.format(args_unit))
print('# Minibatch-size: {}'.format(args_batchsize))
print('# epoch: {}'.format(args_epoch))
print('')

# Set up a neural network to train
# Classifier reports softmax cross entropy loss and accuracy at every
# iteration, which will be used by the PrintReport extension below.
model = MNIST()
if args_gpu >= 0:
    # Make a specified GPU current
    chainer.backends.cuda.get_device_from_id(args_gpu).use()
    model.to_gpu()  # Copy the model to the GPU

# Setup an optimizer
optimizer = chainer.optimizers.Adam(alpha=0.0001)
optimizer.setup(model)

GPU: 0
# unit: 100
# Minibatch-size: 100
# epoch: 3



<H3>学習データであるMNISTデータセット(またはFashion MNIST)の読み込み。初回は多少時間がかかります。。</H3>

In [0]:
# Load the MNIST dataset
train, test = chainer.datasets.get_mnist() # handwritten character
# train, test = chainer.datasets.get_fashion_mnist() # fashion items
train = chainer.datasets.SubDataset(train, 0, 60000)

train_iter = chainer.iterators.SerialIterator(train, args_batchsize)
test_iter = chainer.iterators.SerialIterator(test, args_batchsize,
                                                 repeat=False, shuffle=False)

<H3>MNISTデータの表示。このような画像を５万枚用いて学習します。認識精度のチェックには残り１万枚を用います。</H3>

<h3>ニューラルネットワークの学習を行います。GPUを用いて高速に学習ができます。</h3>

In [112]:
max_epoch = args_epoch
start_time = time.time()

while train_iter.epoch < max_epoch:
    train_accuracies = []
    # ---------- One iteration of the training loop ----------
    train_batch = train_iter.next()
    image_train, target_train = concat_examples(train_batch, args_gpu)

    # Calculate the prediction of the network
    prediction_train = model(image_train)

    # Calculate the loss with softmax_cross_entropy
    loss = F.softmax_cross_entropy(prediction_train, target_train)
    accuracy = F.accuracy(prediction_train, target_train)
    accuracy.to_cpu()
    train_accuracies.append(accuracy.data)
    # Calculate the gradients in the network
    model.cleargrads()
    loss.backward()

    # Update all the trainable parameters
    optimizer.update()
    # --------------------- until here ---------------------

    # Check the validation accuracy of prediction after every epoch
    if train_iter.is_new_epoch:  # If this iteration is the final iteration of the current epoch

        # Display the training loss
        print('epoch:{:02d} train_loss:{:.04f} '.format(
                train_iter.epoch, float(to_cpu(loss.data))), end='')
        print('train_accuracy:{:.04f} '.format(
            np.mean(train_accuracies)),end='')

        test_losses = []
        test_accuracies = []
        while True:
            test_batch = test_iter.next()
            image_test, target_test = concat_examples(test_batch, args_gpu)

            # Forward the test data
            prediction_test = model(image_test)

            # Calculate the loss
            loss_test = F.softmax_cross_entropy(prediction_test, target_test)
            test_losses.append(to_cpu(loss_test.data))

            # Calculate the accuracy
            accuracy = F.accuracy(prediction_test, target_test)
            accuracy.to_cpu()
            test_accuracies.append(accuracy.data)

            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
                np.mean(test_losses), np.mean(test_accuracies)))
        
end_time = time.time()
print("------------------------------------------------------------")
print("Training complete")
chainer.serializers.save_npz('mnist.model',model)

print('accuracy:{:.04f}'.format(np.mean(test_accuracies)))
print('time cost:{:.04f}'.format(end_time-start_time), 's')


epoch:01 train_loss:0.4011 train_accuracy:0.9200 val_loss:0.3844 val_accuracy:0.9009
epoch:02 train_loss:0.2753 train_accuracy:0.9200 val_loss:0.2825 val_accuracy:0.9200
epoch:03 train_loss:0.3101 train_accuracy:0.9000 val_loss:0.2425 val_accuracy:0.9325
------------------------------------------------------------
Training complete
accuracy:0.9325
time cost:7.5639 s
